##### !pip install pyautogui openpyxl

!pip install pywinauto

### !pip install pywinauto

#### import pyautogui

# 화면 크기 확인
screen_width, screen_height = pyautogui.size()

# 좌표를 상대 비율로 계산
x_ratio = 0.3  # '받는 사람' 입력란의 X 비율 (예: 화면의 30%)
y_ratio = 0.4  # '받는 사람' 입력란의 Y 비율 (예: 화면의 40%)

x = int(screen_width * x_ratio)
y = int(screen_height * y_ratio)

# 확인
print(f"계산된 좌표: {x}, {y}")
pyautogui.click(x=x, y=y)

## pip install keyboard pyautogui pandas

In [108]:
pip install keyboard

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# CSV 파일 경로
FILE_PATH = r'D:\체납\11월체납활동\체납문자발송.cs'

# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  # 각 번호 입력 후 딜레이
DELAY_BETWEEN_TABS = 0.1  # 탭키 입력 후 딜레이


def load_data():
    """
    CSV 파일에서 데이터를 로드하고 번호를 11자리로 맞춤
    """
    try:
        data = pd.read_csv(FILE_PATH, encoding='euc-kr')
        data['문자발송용'] = data['문자발송용'].astype(str).apply(lambda x: x.zfill(11))
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"CSV 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()


def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    # 불러온 번호 테이블 갱신
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))  # 원래 no 유지

    # 등록 완료된 번호 테이블 갱신
    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))


def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        # 번호 입력
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  # 딜레이 최소화
        pyautogui.press("tab")  # 탭키로 다음 입력란으로 이동
        time.sleep(DELAY_BETWEEN_TABS)  # 탭키 이동 후 딜레이

        # 등록 완료 처리
        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")


def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    # 좌표 설정
    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)  # 5초 대기
        start_position = pyautogui.position()  # 현재 마우스 위치 가져오기

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    # 시작 인덱스 및 종료 인덱스 계산
    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    # 입력란 클릭으로 시작 위치 설정
    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    # 100개 단위로 등록 진행
    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['문자발송용'])

    # 진행된 범위 표시
    start_index = end_index  # 다음 실행 시 시작 인덱스 업데이트
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")


def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("900x600")

    # 타이틀
    title_label = tk.Label(root, text="휴대폰 번호 등록 시스템", font=("Arial", 16, "bold"))
    title_label.pack(pady=10)

    # 테이블 프레임
    table_frame = tk.Frame(root)
    table_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    # 왼쪽 테이블 (불러온 번호)
    left_frame = tk.Frame(table_frame, relief=tk.GROOVE, bd=2)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 12, "bold"))
    left_label.pack(pady=5)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(left_frame, columns=("no", "number"), show="headings", height=20, yscrollcommand=left_scroll.set)
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="no")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=200, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True)

    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))  # 원래 no 유지

    # 오른쪽 테이블 (등록 완료 번호)
    right_frame = tk.Frame(table_frame, relief=tk.GROOVE, bd=2)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 12, "bold"))
    right_label.pack(pady=5)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(right_frame, columns=("no", "number"), show="headings", height=20, yscrollcommand=right_scroll.set)
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="no")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=200, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True)

    # 하단 버튼
    button_frame = tk.Frame(root)
    button_frame.pack(fill=tk.X, padx=10, pady=10)

    manual_button = tk.Button(button_frame, text="실행", command=lambda: manual_registration(data), font=("Arial", 12), bg="green", fg="white")
    manual_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(button_frame, text="종료", command=root.destroy, font=("Arial", 12), bg="red", fg="white")
    exit_button.pack(side=tk.RIGHT, padx=10)

    root.mainloop()


if __name__ == "__main__":
    data = load_data()  # 데이터 로드
    create_gui(data)  # GUI 생성

번호 '01068076991' 등록 완료
번호 '01067208150' 등록 완료
번호 '01045030724' 등록 완료
번호 '01094869639' 등록 완료
번호 '01028621609' 등록 완료
번호 '01050001050' 등록 완료
번호 '01021665562' 등록 완료
번호 '01050271011' 등록 완료
번호 '01024773676' 등록 완료
번호 '01049726389' 등록 완료
번호 '01066314624' 등록 완료
번호 '01027255320' 등록 완료
번호 '01091910302' 등록 완료
번호 '01054207995' 등록 완료
번호 '01075459110' 등록 완료
번호 '01085556882' 등록 완료
번호 '01056140547' 등록 완료
번호 '01037778824' 등록 완료
번호 '01064143382' 등록 완료
번호 '01082647533' 등록 완료
번호 '01057023969' 등록 완료
번호 '01086606559' 등록 완료
번호 '01065116065' 등록 완료
번호 '01046605085' 등록 완료
번호 '01044143940' 등록 완료
번호 '01028826161' 등록 완료
번호 '01020744112' 등록 완료
번호 '01040893588' 등록 완료
번호 '01037192988' 등록 완료
번호 '01088483109' 등록 완료
번호 '01075588968' 등록 완료
번호 '01033804959' 등록 완료
번호 '01071943259' 등록 완료
번호 '01029955709' 등록 완료
번호 '01033377504' 등록 완료
번호 '01083871005' 등록 완료
번호 '01066601196' 등록 완료
번호 '01023004298' 등록 완료
번호 '01037855328' 등록 완료
번호 '01074249193' 등록 완료
번호 '01085356737' 등록 완료
번호 '01099532547' 등록 완료
번호 '01067651833' 등록 완료
번호 '0102667

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# CSV 파일 경로
FILE_PATH = r'D:\체납\11월체납활동\체납단체CMS미납대상문자발송20241128.cs'

# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  # 각 번호 입력 후 딜레이
DELAY_BETWEEN_TABS = 0.1  # 탭키 입력 후 딜레이


def load_data():
    """
    CSV 파일에서 데이터를 로드하고 번호를 11자리로 맞춤
    """
    try:
        data = pd.read_csv(FILE_PATH, encoding='euc-kr')
        data['문자발송용'] = data['문자발송용'].astype(str).apply(lambda x: x.zfill(11))
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"CSV 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()


def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    # 불러온 번호 테이블 갱신
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))  # 원래 no 유지

    # 등록 완료된 번호 테이블 갱신
    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))


def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        # 번호 입력
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  # 딜레이 최소화
        pyautogui.press("tab")  # 탭키로 다음 입력란으로 이동
        time.sleep(DELAY_BETWEEN_TABS)  # 탭키 이동 후 딜레이

        # 등록 완료 처리
        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")


def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    # 좌표 설정
    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)  # 5초 대기
        start_position = pyautogui.position()  # 현재 마우스 위치 가져오기

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    # 시작 인덱스 및 종료 인덱스 계산
    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    # 입력란 클릭으로 시작 위치 설정
    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    # 100개 단위로 등록 진행
    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['문자발송용'])

    # 진행된 범위 표시
    start_index = end_index  # 다음 실행 시 시작 인덱스 업데이트
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")


def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("900x600")

    # 타이틀
    title_label = tk.Label(root, text="휴대폰 번호 등록 시스템", font=("Arial", 16, "bold"))
    title_label.pack(pady=10)

    # 테이블 프레임
    table_frame = tk.Frame(root)
    table_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    # 왼쪽 테이블 (불러온 번호)
    left_frame = tk.Frame(table_frame, relief=tk.GROOVE, bd=2)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 12, "bold"))
    left_label.pack(pady=5)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(left_frame, columns=("no", "number"), show="headings", height=20, yscrollcommand=left_scroll.set)
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="no")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=200, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True)

    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))  # 원래 no 유지

    # 오른쪽 테이블 (등록 완료 번호)
    right_frame = tk.Frame(table_frame, relief=tk.GROOVE, bd=2)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 12, "bold"))
    right_label.pack(pady=5)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(right_frame, columns=("no", "number"), show="headings", height=20, yscrollcommand=right_scroll.set)
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="no")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=200, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True)

    # 하단 버튼
    button_frame = tk.Frame(root)
    button_frame.pack(fill=tk.X, padx=10, pady=10)

    manual_button = tk.Button(button_frame, text="실행", command=lambda: manual_registration(data), font=("Arial", 12), bg="green", fg="white")
    manual_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(button_frame, text="종료", command=root.destroy, font=("Arial", 12), bg="red", fg="white")
    exit_button.pack(side=tk.RIGHT, padx=10)

    root.mainloop()


if __name__ == "__main__":
    data = load_data()  # 데이터 로드
    create_gui(data)  # GUI 생성

## cs파일문자

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# CSV 파일 경로
FILE_PATH = r'D:\체납\이화영\체납문자발송대상0318.cs'

# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  # 각 번호 입력 후 딜레이
DELAY_BETWEEN_TABS = 0.1  # 탭키 입력 후 딜레이


def load_data():
    """
    CSV 파일에서 데이터를 로드하고 번호를 11자리로 맞춤
    """
    try:
        data = pd.read_csv(FILE_PATH, encoding='euc-kr')
        data['문자발송용'] = data['문자발송용'].astype(str).apply(lambda x: x.zfill(11))
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"CSV 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()


def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    # 불러온 번호 테이블 갱신
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))

    # 등록 완료된 번호 테이블 갱신
    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))


def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  # 딜레이 최소화
        pyautogui.press("tab")  # 탭키로 다음 입력란으로 이동
        time.sleep(DELAY_BETWEEN_TABS)  # 탭키 이동 후 딜레이

        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")


def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)  # 5초 대기
        start_position = pyautogui.position()

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['문자발송용'])

    start_index = end_index
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")


def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("1000x700")
    root.configure(bg="#f8f9fa")

    # 타이틀
    title_label = tk.Label(
        root, text="휴대폰 번호 등록 시스템", font=("Arial", 20, "bold"), bg="#343a40", fg="#ffffff", pady=10
    )
    title_label.pack(fill=tk.X)

    # 테이블 프레임
    table_frame = tk.Frame(root, bg="#f8f9fa")
    table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

    # 왼쪽 테이블 (불러온 번호)
    left_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    left_label.pack(pady=10)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(
        left_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=left_scroll.set,
    )
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="No")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=250, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 오른쪽 테이블 (등록 완료 번호)
    right_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    right_label.pack(pady=10)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(
        right_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=right_scroll.set,
    )
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="No")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=250, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 하단 버튼
    button_frame = tk.Frame(root, bg="#f8f9fa")
    button_frame.pack(fill=tk.X, pady=10)

    execute_button = tk.Button(
        button_frame,
        text="실행",
        command=lambda: manual_registration(data),
        font=("Arial", 14, "bold"),
        bg="#28a745",
        fg="#ffffff",
        width=15,
    )
    execute_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(
        button_frame,
        text="종료",
        command=root.destroy,
        font=("Arial", 14, "bold"),
        bg="#dc3545",
        fg="#ffffff",
        width=15,
    )
    exit_button.pack(side=tk.RIGHT, padx=10)

    update_tables(data)
    root.mainloop()


if __name__ == "__main__":
    data = load_data()
    create_gui(data)

번호 '01093676116' 등록 완료
번호 '01072237074' 등록 완료
번호 '01062985886' 등록 완료
번호 '01024318707' 등록 완료
번호 '01051437414' 등록 완료
번호 '01057800826' 등록 완료
번호 '01076460164' 등록 완료
번호 '01063748122' 등록 완료
번호 '01052304009' 등록 완료
번호 '01033680332' 등록 완료
번호 '01053836863' 등록 완료
번호 '01085356737' 등록 완료
번호 '01049379765' 등록 완료
번호 '01037328218' 등록 완료
번호 '01066422479' 등록 완료
번호 '01053216600' 등록 완료
번호 '01037089135' 등록 완료
번호 '01021665562' 등록 완료
번호 '01091362557' 등록 완료
번호 '01020744112' 등록 완료
번호 '01052091029' 등록 완료
번호 '01055207131' 등록 완료
번호 '01063850963' 등록 완료
번호 '01027174731' 등록 완료
번호 '01039598317' 등록 완료
번호 '01035310138' 등록 완료
번호 '01044680315' 등록 완료
번호 '01024773676' 등록 완료
번호 '01055168126' 등록 완료
번호 '01024472959' 등록 완료
번호 '01031145486' 등록 완료
번호 '01075588968' 등록 완료
번호 '01075124019' 등록 완료
번호 '01062365182' 등록 완료
번호 '01072670101' 등록 완료
번호 '01063068284' 등록 완료
번호 '01042008307' 등록 완료
번호 '01066472499' 등록 완료
번호 '01065190915' 등록 완료
번호 '01063734783' 등록 완료
번호 '01092047220' 등록 완료
번호 '01067957312' 등록 완료
번호 '01087178738' 등록 완료
번호 '0103875

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# CSV 파일 경로
FILE_PATH = r'D:\시설\정지\일일정지\20250114_sp별정지사유등록문자발송2.cs'



# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  # 각 번호 입력 후 딜레이
DELAY_BETWEEN_TABS = 0.1  # 탭키 입력 후 딜레이


def load_data():
    """
    CSV 파일에서 데이터를 로드하고 번호를 11자리로 맞춤
    """
    try:
        data = pd.read_csv(FILE_PATH, encoding='euc-kr')
        data['문자발송용'] = data['문자발송용'].astype(str).apply(lambda x: x.zfill(11))
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"CSV 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()


def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    # 불러온 번호 테이블 갱신
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['문자발송용'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['문자발송용']))

    # 등록 완료된 번호 테이블 갱신
    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))


def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  # 딜레이 최소화
        pyautogui.press("tab")  # 탭키로 다음 입력란으로 이동
        time.sleep(DELAY_BETWEEN_TABS)  # 탭키 이동 후 딜레이

        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")


def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)  # 5초 대기
        start_position = pyautogui.position()

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['문자발송용'])

    start_index = end_index
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")


def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("1000x700")
    root.configure(bg="#f8f9fa")

    # 타이틀
    title_label = tk.Label(
        root, text="휴대폰 번호 등록 시스템", font=("Arial", 20, "bold"), bg="#343a40", fg="#ffffff", pady=10
    )
    title_label.pack(fill=tk.X)

    # 테이블 프레임
    table_frame = tk.Frame(root, bg="#f8f9fa")
    table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

    # 왼쪽 테이블 (불러온 번호)
    left_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    left_label.pack(pady=10)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(
        left_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=left_scroll.set,
    )
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="No")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=250, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 오른쪽 테이블 (등록 완료 번호)
    right_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    right_label.pack(pady=10)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(
        right_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=right_scroll.set,
    )
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="No")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=250, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True, padx=10)

    # 하단 버튼
    button_frame = tk.Frame(root, bg="#f8f9fa")
    button_frame.pack(fill=tk.X, pady=10)

    execute_button = tk.Button(
        button_frame,
        text="실행",
        command=lambda: manual_registration(data),
        font=("Arial", 14, "bold"),
        bg="#28a745",
        fg="#ffffff",
        width=15,
    )
    execute_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(
        button_frame,
        text="종료",
        command=root.destroy,
        font=("Arial", 14, "bold"),
        bg="#dc3545",
        fg="#ffffff",
        width=15,
    )
    exit_button.pack(side=tk.RIGHT, padx=10)

    update_tables(data)
    root.mainloop()


if __name__ == "__main__":
    data = load_data()
    create_gui(data)

번호 '01071954890' 등록 완료
번호 '01028891608' 등록 완료
번호 '01032280237' 등록 완료
번호 '01040048898' 등록 완료
번호 '01042613797' 등록 완료
번호 '01032418520' 등록 완료
번호 '01088691347' 등록 완료
번호 '01044302393' 등록 완료
번호 '01058805501' 등록 완료
번호 '01037165315' 등록 완료
번호 '01020805063' 등록 완료
번호 '01071638361' 등록 완료
번호 '01099030707' 등록 완료
번호 '01043699112' 등록 완료
번호 '01044575009' 등록 완료
번호 '01051956329' 등록 완료
번호 '01092090419' 등록 완료
번호 '01091888887' 등록 완료
번호 '01053514624' 등록 완료
번호 '01099791104' 등록 완료
번호 '01079005280' 등록 완료
번호 '01027631951' 등록 완료
번호 '01034480466' 등록 완료
번호 '01086344514' 등록 완료
번호 '01066497514' 등록 완료
번호 '01054737057' 등록 완료
번호 '01099417372' 등록 완료
번호 '01043931053' 등록 완료
번호 '01071251116' 등록 완료
번호 '01043250811' 등록 완료
번호 '01087418315' 등록 완료
번호 '01067367431' 등록 완료
번호 '01076850112' 등록 완료
번호 '01031730219' 등록 완료
번호 '01067234755' 등록 완료
번호 '01051299293' 등록 완료
번호 '01044456171' 등록 완료
번호 '01080102259' 등록 완료
번호 '01026621073' 등록 완료
번호 '01089193323' 등록 완료
번호 '01062511326' 등록 완료
번호 '01046936866' 등록 완료
번호 '01022703685' 등록 완료
번호 '0108867

In [ ]:
D:\시설\정지\일일정지\20250114_sp별정지사유등록문자발송2.cs

## 문자발송 엑셀파일 엑세스에서 암호해제하여 사용

In [13]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import pyautogui
import time

# 엑셀 파일 경로
FILE_PATH = r'D:\체납\이화영\체납문자발송대상0318.xlsx'

# 전역 변수
start_position = None  # 첫 번째 클릭 위치 (등록 시작점)
registered_numbers = []  # 등록 완료된 번호 리스트
start_index = 0  # 현재 등록 시작 인덱스 (100개 단위)

# 딜레이 설정 (초)
DELAY_BETWEEN_INPUTS = 0.05  
DELAY_BETWEEN_TABS = 0.1  

def load_data():
    """
    엑셀 파일에서 B2부터 데이터를 불러오고, 전화번호를 11자리로 변환
    """
    try:
        data = pd.read_excel(FILE_PATH, engine='openpyxl', usecols="B", skiprows=1)  # B열만 불러오기 (B2부터)
        data.columns = ["전화번호"]  # 컬럼명 설정
        data['전화번호'] = data['전화번호'].astype(str).str.zfill(11)  # 11자리 변환
        data.reset_index(drop=True, inplace=True)
        return data
    except FileNotFoundError:
        messagebox.showerror("파일 오류", f"엑셀 파일을 찾을 수 없습니다: {FILE_PATH}")
        return pd.DataFrame()
    except Exception as e:
        messagebox.showerror("데이터 오류", f"데이터를 불러오는 중 오류가 발생했습니다: {e}")
        return pd.DataFrame()

def update_tables(data):
    """
    테이블 업데이트: 불러온 번호와 등록 완료된 번호를 업데이트
    """
    left_table.delete(*left_table.get_children())
    remaining_numbers = data[~data['전화번호'].isin(registered_numbers)]
    for idx, row in remaining_numbers.iterrows():
        left_table.insert("", "end", values=(row.name + 1, row['전화번호']))

    registered_table.delete(*registered_table.get_children())
    for idx, number in enumerate(registered_numbers, start=1):
        registered_table.insert("", "end", values=(idx, number))

def execute_registration(number):
    """
    번호 등록 실행: 지정된 좌표에 값을 입력하고 탭키로 이동
    """
    try:
        pyautogui.write(number)
        time.sleep(DELAY_BETWEEN_INPUTS)  
        pyautogui.press("tab")  
        time.sleep(DELAY_BETWEEN_TABS)

        registered_numbers.append(number)
        print(f"번호 '{number}' 등록 완료")
    except Exception as e:
        messagebox.showerror("등록 오류", f"번호 '{number}' 등록 실패: {e}")

def manual_registration(data):
    """
    수동 등록: 좌표 설정 후 100개 단위로 입력
    """
    global start_position, start_index

    if not start_position:
        messagebox.showinfo("좌표 설정", "5초 안에 '받는 사람' 입력란을 클릭하세요.")
        time.sleep(5)
        start_position = pyautogui.position()

        if not start_position:
            messagebox.showwarning("좌표 설정 실패", "입력란 좌표를 설정하지 못했습니다. 다시 시도하세요.")
            return

    x, y = start_position

    end_index = start_index + 100
    if start_index >= len(data):
        messagebox.showinfo("완료", "더 이상 등록할 번호가 없습니다.")
        return

    pyautogui.click(x=x, y=y)
    time.sleep(0.2)

    for idx, row in data[start_index:end_index].iterrows():
        execute_registration(row['전화번호'])

    start_index = end_index
    update_tables(data)
    messagebox.showinfo("완료", f"{start_index - 99}번부터 {start_index}번까지 등록이 완료되었습니다!")

def create_gui(data):
    """
    GUI 생성 및 구성
    """
    global left_table, registered_table

    root = tk.Tk()
    root.title("휴대폰 번호 등록 시스템")
    root.geometry("1000x700")
    root.configure(bg="#f8f9fa")

    title_label = tk.Label(
        root, text="휴대폰 번호 등록 시스템", font=("Arial", 20, "bold"), bg="#343a40", fg="#ffffff", pady=10
    )
    title_label.pack(fill=tk.X)

    table_frame = tk.Frame(root, bg="#f8f9fa")
    table_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

    left_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10)

    left_label = tk.Label(left_frame, text="불러온 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    left_label.pack(pady=10)

    left_scroll = tk.Scrollbar(left_frame, orient="vertical")
    left_table = ttk.Treeview(
        left_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=left_scroll.set,
    )
    left_scroll.config(command=left_table.yview)
    left_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    left_table.heading("no", text="No")
    left_table.heading("number", text="휴대폰번호")
    left_table.column("no", width=50, anchor="center")
    left_table.column("number", width=250, anchor="center")
    left_table.pack(fill=tk.BOTH, expand=True, padx=10)

    right_frame = tk.Frame(table_frame, bg="#ffffff", bd=2, relief=tk.RIDGE)
    right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10)

    right_label = tk.Label(right_frame, text="등록완료된 휴대폰번호", font=("Arial", 14, "bold"), bg="#ffffff")
    right_label.pack(pady=10)

    right_scroll = tk.Scrollbar(right_frame, orient="vertical")
    registered_table = ttk.Treeview(
        right_frame,
        columns=("no", "number"),
        show="headings",
        height=20,
        yscrollcommand=right_scroll.set,
    )
    right_scroll.config(command=registered_table.yview)
    right_scroll.pack(side=tk.RIGHT, fill=tk.Y)

    registered_table.heading("no", text="No")
    registered_table.heading("number", text="휴대폰번호")
    registered_table.column("no", width=50, anchor="center")
    registered_table.column("number", width=250, anchor="center")
    registered_table.pack(fill=tk.BOTH, expand=True, padx=10)

    button_frame = tk.Frame(root, bg="#f8f9fa")
    button_frame.pack(fill=tk.X, pady=10)

    execute_button = tk.Button(
        button_frame,
        text="실행",
        command=lambda: manual_registration(data),
        font=("Arial", 14, "bold"),
        bg="#28a745",
        fg="#ffffff",
        width=15,
    )
    execute_button.pack(side=tk.LEFT, padx=10)

    exit_button = tk.Button(
        button_frame,
        text="종료",
        command=root.destroy,
        font=("Arial", 14, "bold"),
        bg="#dc3545",
        fg="#ffffff",
        width=15,
    )
    exit_button.pack(side=tk.RIGHT, padx=10)

    update_tables(data)
    root.mainloop()

if __name__ == "__main__":
    data = load_data()
    create_gui(data)

번호 '01024063982' 등록 완료
번호 '01049962015' 등록 완료
번호 '01062365182' 등록 완료
번호 '01066666462' 등록 완료
번호 '01089942600' 등록 완료
번호 '01034338058' 등록 완료
번호 '01033377504' 등록 완료
번호 '01043151684' 등록 완료
번호 '01056942596' 등록 완료
번호 '01082786397' 등록 완료
번호 '01052091029' 등록 완료
번호 '01026678831' 등록 완료
번호 '01033059462' 등록 완료
번호 '01076452801' 등록 완료
번호 '01029532711' 등록 완료
번호 '01028829129' 등록 완료
번호 '01072878135' 등록 완료
번호 '01077632363' 등록 완료
번호 '01072878153' 등록 완료
번호 '01029090453' 등록 완료
번호 '01072336187' 등록 완료
번호 '01092805360' 등록 완료
번호 '01052473288' 등록 완료
번호 '01030069629' 등록 완료
번호 '01066000465' 등록 완료
번호 '01089502855' 등록 완료
번호 '01090711876' 등록 완료
번호 '01042110424' 등록 완료
번호 '01068567707' 등록 완료
번호 '01089510412' 등록 완료
번호 '01075122785' 등록 완료
번호 '01063344539' 등록 완료
번호 '01051740932' 등록 완료
번호 '01099821672' 등록 완료
번호 '01034762622' 등록 완료
번호 '01022862271' 등록 완료
번호 '01064429021' 등록 완료
번호 '01077329144' 등록 완료
번호 '01086543929' 등록 완료
번호 '01033836625' 등록 완료
번호 '01095243002' 등록 완료
번호 '01086524429' 등록 완료
번호 '01056645115' 등록 완료
번호 '0109113